In [1]:
param_market = 'DE'

In [2]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./menu.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(666):
            try:
                if param_market == 'US':
                    url = 'https://www.ebay.com/itm/' + input_.loc[a, 'Item_Number'] + '?_ul=US&_stpos=91710&orig_cvip=true'
                elif param_market == 'DE':
                    url = 'https://www.ebay.de/itm/' + input_.loc[a, 'Item_Number'] + '?_ul=DE&_stpos=10115&orig_cvip=true'
                elif param_market == 'AU':
                    url = 'https://www.ebay.com.au/itm/' + input_.loc[a, 'Item_Number'] + '?_ul=AU&_stpos=2019&orig_cvip=true'
                elif param_market == 'UK':
                    url = 'https://www.ebay.co.uk/itm/' + input_.loc[a, 'Item_Number'] + '?_ul=GB&_stpos=BS81QU&orig_cvip=true'

                resp = requests.get(url,
                                    #headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                json_data = json.loads(resp.split('$MC=(window.$MC||[]).concat(')[1].split(']}]})</script></body></html>')[0] + ']}]}')
                
                # = = = = = = = = = = = = = = =
                
                market_id = json_data['o']['g']['marketplaceId'].strip()
                
                # = = = = = = = = = = = = = = =
                
                country = json_data['o']['g']['country'].strip()

                # = = = = = = = = = = = = = = =
                
                language = json_data['o']['g']['language'].strip()

                # = = = = = = = = = = = = = = =
                
                locale = json_data['o']['g']['locale'].strip()

                # = = = = = = = = = = = = = = =
                
                item_id = json_data['o']['g']['itemId'].strip()

                # = = = = = = = = = = = = = = =
                
                if param_market == 'US':
                    if market_id != 'EBAY-US' or country != 'US' or language != 'en' or locale != 'en-US' or item_id != input_.loc[a, 'Item_Number']:
                        continue
                if param_market == 'AU':
                    if market_id != 'EBAY-AU' or country != 'AU' or language != 'en' or locale != 'en-AU' or item_id != input_.loc[a, 'Item_Number']:
                        continue
                elif param_market == 'UK':
                    if market_id != 'EBAY-GB' or country != 'GB' or language != 'en' or locale != 'en-GB' or item_id != input_.loc[a, 'Item_Number']:
                        continue
                    
                # = = = = = = = = = = = = = = =
                
                specification = json_data['o']['w'][0][2]['model']['modules']['ABOUT_THIS_ITEM']['sections']['features']['dataItems']

                # = = = = = = = = = = = = = = =
                
                if param_market in ['US', 'AU', 'UK']:
                    if 'VEHICLE_SELECTION' in json_data['o']['w'][0][2]['model']['modules']:
                        category_id = json_data['o']['w'][0][2]['model']['modules']['VEHICLE_SELECTION']['callToActions'][0]['action']['params']['categoryId'].strip()
                        # print(json_data['o']['w'][0][2]['model']['modules']['COMPATIBILITY_TABLE']['paginatedTable']['title']['textSpans'][0]['text'].strip())
                        vehicle_page = json_data['o']['w'][0][2]['model']['modules']['COMPATIBILITY_TABLE']['paginatedTable']['pagination']['itemsPerPage']['label']['textSpans'][0]['text'].split('of')[1].strip()
                    else:
                        category_id = ''
                        vehicle_page = ''
                elif param_market == 'DE':
                    if 'MOTORS_TAB' in json_data['o']['w'][0][2]['model']['modules']:
                        if 'vehicleHsnTsnLookupTab' in json_data['o']['w'][0][2]['model']['modules']['MOTORS_TAB']['tabContent']:
                            category_id = json_data['o']['w'][0][2]['model']['modules']['MOTORS_TAB']['tabContent']['vehicleHsnTsnLookupTab']['callToActions'][0]['action']['params']['categoryId'].strip()
                        elif 'vehicleSelectionTab' in json_data['o']['w'][0][2]['model']['modules']['MOTORS_TAB']['tabContent']:
                            category_id = json_data['o']['w'][0][2]['model']['modules']['MOTORS_TAB']['tabContent']['vehicleSelectionTab']['callToActions'][0]['action']['params']['categoryId'].strip()
                        else:
                            print('fuck')
                        #print(json_data['o']['w'][0][2]['model']['modules']['COMPATIBILITY_TABLE']['paginatedTable']['title']['textSpans'][0]['text'].strip())
                        vehicle_page = json_data['o']['w'][0][2]['model']['modules']['COMPATIBILITY_TABLE']['paginatedTable']['pagination']['itemsPerPage']['label']['textSpans'][0]['text'].split('von')[1].strip()
                    else:
                        category_id = ''
                        vehicle_page = ''
                        
                # = = = = = = = = = = = = = = =
                
                title = json_data['o']['w'][0][2]['model']['modules']['TITLE']['mainTitle']['textSpans'][0]['text'].strip()

                # = = = = = = = = = = = = = = =
                
                if 'subTitle' in json_data['o']['w'][0][2]['model']['modules']['TITLE']:
                    subtitle = json_data['o']['w'][0][2]['model']['modules']['TITLE']['subTitle']['textSpans'][0]['text'].strip()
                else:
                    subtitle = ''
                    
                # = = = = = = = = = = = = = = =
                
                if 'binModel' in json_data['o']['w'][0][2]['model']['modules']['BUY_BOX']:
                    price = json_data['o']['w'][0][2]['model']['modules']['BUY_BOX']['binModel']['price']['textSpans'][0]['text'].strip()
                elif 'bidPrice' in json_data['o']['w'][0][2]['model']['modules']['BUY_BOX']:
                    price = json_data['o']['w'][0][2]['model']['modules']['BUY_BOX']['bidPrice']['textSpans'][0]['text'].strip()

                # = = = = = = = = = = = = = = =
                
                if 'QUANTITY' in json_data['o']['w'][0][2]['model']['modules']:
                    available = ''
                    sold = ''
                    list_text = json_data['o']['w'][0][2]['model']['modules']['QUANTITY']['availabilitySignal']['textSpans']
                    for text in list_text:
                        if param_market in ['US', 'AU', 'UK']:
                            if 'available' in text['text']:
                                available = text['text'].strip()
                            elif 'sold' in text['text']:
                                sold = text['text'].strip()
                        elif param_market == 'DE':
                            if 'verfügbar' in text['text']:
                                available = text['text'].strip()
                            elif 'verkauft' in text['text']:
                                sold = text['text'].strip()
                else:
                    available = ''
                    sold = ''
                    
                # = = = = = = = = = = = = = = =
                
                if 'mediaList' not in json_data['o']['w'][0][2]['model']['modules']['PICTURE']:
                    src = ''
                elif 'largeImg' in json_data['o']['w'][0][2]['model']['modules']['PICTURE']['mediaList'][0]['image']:
                    src = json_data['o']['w'][0][2]['model']['modules']['PICTURE']['mediaList'][0]['image']['largeImg']['URL'].strip()
                else:
                    src = json_data['o']['w'][0][2]['model']['modules']['PICTURE']['mediaList'][0]['image']['originalImg']['URL'].strip()

                # = = = = = = = = = = = = = = =
                
                if 'action' in json_data['o']['w'][0][2]['model']['modules']['ITEM_DESCRIPTION_MIN_VIEW_MODULE']['sections'][0]:
                    description_url = json_data['o']['w'][0][2]['model']['modules']['ITEM_DESCRIPTION_MIN_VIEW_MODULE']['sections'][0]['action']['URL'].strip()
                else:
                    description_url = json_data['o']['w'][0][2]['model']['modules']['ITEM_DESCRIPTION_MIN_VIEW_MODULE']['sections'][0]['dataItems'][0]['action']['URL']

                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'序号': int(input_.loc[a, 'No.']),
                                         'Item_Number': input_.loc[a, 'Item_Number'],
                                         'Title': title,
                                         'Subtitle': subtitle,
                                         'Price': price,
                                         'Available': available,
                                         'Sold': sold,
                                         'Vehicle_1': '',
                                         'Vehicle_2': '',
                                         'Pic': '',
                                         'Url': url,
                                         'Description_Url': description_url,
                                         'Src': src,
                                         'Vehicle_Page': vehicle_page,
                                         'Category_Id': category_id,
                                         'Market_Id': market_id,
                                         'Country': country,
                                         'Language': language,
                                         'Locale': locale}])

                dict_name = {}
                if param_market in ['US', 'AU', 'UK']:
                    condition = 'Condition'
                elif param_market == 'DE':
                    condition = 'Artikelzustand'

                for name_value in specification:
                    name = specification[name_value]['labels'][0]['textSpans'][0]['text'].strip()
                    if name == condition:
                        df_temp[condition] = specification[name_value]['values'][0]['textualDisplays'][0]['textSpans'][0]['text'].strip() if 'textualDisplays' in specification[name_value]['values'][0] else specification[name_value]['values'][0]['textSpans'][0]['text'].strip()
                    else:
                        name = name.lower()
                        if name in dict_name:
                            dict_name[name] += 1
                        else:
                            dict_name[name] = 1
                        df_temp[name if dict_name[name] == 1 else name + '-' + str(dict_name[name])] = specification[name_value]['values'][0]['textualDisplays'][0]['textSpans'][0]['text'].strip() if 'textualDisplays' in specification[name_value]['values'][0] else specification[name_value]['values'][0]['textSpans'][0]['text'].strip()

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'No.': int(input_.loc[a, 'No.']),
                                     'Item_Number': input_.loc[a, 'Item_Number']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Item_Number'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['序号'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./item-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['No.'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./item_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：1440

[ok] - 386928426035
[尝试次数：1] - [剩余数量：1420] - [当前时间：21:07:54]

[ok] - 404782291350
[尝试次数：1] - [剩余数量：1419] - [当前时间：21:07:54]

[ok] - 315071412290
[尝试次数：1] - [剩余数量：1418] - [当前时间：21:07:54]

[ok] - 315281836881
[尝试次数：1] - [剩余数量：1417] - [当前时间：21:07:55]

[ok] - 314966582305
[尝试次数：1] - [剩余数量：1416] - [当前时间：21:07:55]

[ok] - 315145650917
[尝试次数：1] - [剩余数量：1415] - [当前时间：21:07:55]

[ok] - 404549041123
[尝试次数：1] - [剩余数量：1414] - [当前时间：21:07:55]

[ok] - 404817618477
[尝试次数：1] - [剩余数量：1413] - [当前时间：21:07:55]

[ok] - 404912782489
[尝试次数：1] - [剩余数量：1412] - [当前时间：21:07:55]

[ok] - 386917055851
[尝试次数：1] - [剩余数量：1411] - [当前时间：21:07:55]

[ok] - 315281840098
[尝试次数：1] - [剩余数量：1410] - [当前时间：21:07:55]

[ok] - 386916908001
[尝试次数：1] - [剩余数量：1409] - [当前时间：21:07:56]

[ok] - 386205433715
[尝试次数：1] - [剩余数量：1408] - [当前时间：21:07:56]

[ok] - 404912779852
[尝试次数：1] - [剩余数量：1407] - [当前时间：21:07:56]

[ok] - 314966582400
[尝试次数：1] - [剩余数量：1406] - [当前时间：21:07:57]

[ok] - 404951737888
[尝试次数：1] - [剩余数量：1405] - [当前时间：21:07:57]

[ok] - 314754889147
[尝试次数：1] - [剩余数量：1288] - [当前时间：21:08:35]

[ok] - 314879676884
[尝试次数：1] - [剩余数量：1287] - [当前时间：21:08:36]

[ok] - 404761464119
[尝试次数：1] - [剩余数量：1286] - [当前时间：21:08:36]

[ok] - 404624706443
[尝试次数：1] - [剩余数量：1285] - [当前时间：21:08:36]

[ok] - 385736646807
[尝试次数：1] - [剩余数量：1284] - [当前时间：21:08:37]

[ok] - 386924253685
[尝试次数：1] - [剩余数量：1283] - [当前时间：21:08:37]

[ok] - 404722393120
[尝试次数：1] - [剩余数量：1282] - [当前时间：21:08:37]

[ok] - 386903349682
[尝试次数：1] - [剩余数量：1281] - [当前时间：21:08:37]

[ok] - 386824683746
[尝试次数：1] - [剩余数量：1280] - [当前时间：21:08:38]

[ok] - 404565207562
[尝试次数：1] - [剩余数量：1279] - [当前时间：21:08:39]

[ok] - 404722392960
[尝试次数：1] - [剩余数量：1278] - [当前时间：21:08:39]

[ok] - 386338873693
[尝试次数：1] - [剩余数量：1277] - [当前时间：21:08:39]

[ok] - 385579157224
[尝试次数：1] - [剩余数量：1276] - [当前时间：21:08:39]

[ok] - 312605422727
[尝试次数：1] - [剩余数量：1275] - [当前时间：21:08:39]

[ok] - 386577252319
[尝试次数：1] - [剩余数量：1274] - [当前时间：21:08:39]

[ok] - 386975151162
[尝试次数：2] - [剩余数量：1273] - [当前时间：21:08:40]

[ok] - 3

[ok] - 314860895302
[尝试次数：1] - [剩余数量：1155] - [当前时间：21:09:23]

[ok] - 386244349695
[尝试次数：2] - [剩余数量：1154] - [当前时间：21:09:23]

[ok] - 386932430308
[尝试次数：1] - [剩余数量：1153] - [当前时间：21:09:23]

[ok] - 405014000279
[尝试次数：1] - [剩余数量：1152] - [当前时间：21:09:24]

[ok] - 386942077568
[尝试次数：1] - [剩余数量：1151] - [当前时间：21:09:24]

[ok] - 404761234539
[尝试次数：1] - [剩余数量：1150] - [当前时间：21:09:25]

[ok] - 315225923215
[尝试次数：1] - [剩余数量：1149] - [当前时间：21:09:25]

[ok] - 387014449259
[尝试次数：1] - [剩余数量：1148] - [当前时间：21:09:26]

[ok] - 315238974672
[尝试次数：1] - [剩余数量：1147] - [当前时间：21:09:26]

[ok] - 404727532225
[尝试次数：1] - [剩余数量：1146] - [当前时间：21:09:26]

[ok] - 387037510071
[尝试次数：1] - [剩余数量：1145] - [当前时间：21:09:28]

[ok] - 315281832616
[尝试次数：2] - [剩余数量：1144] - [当前时间：21:09:28]

[ok] - 315255947794
[尝试次数：1] - [剩余数量：1143] - [当前时间：21:09:28]

[ok] - 315293338733
[尝试次数：1] - [剩余数量：1142] - [当前时间：21:09:28]

[ok] - 385219731616
[尝试次数：1] - [剩余数量：1141] - [当前时间：21:09:28]

[ok] - 381564339136
[尝试次数：1] - [剩余数量：1140] - [当前时间：21:09:28]

[ok] - 3

[ok] - 386639799626
[尝试次数：1] - [剩余数量：1021] - [当前时间：21:10:06]

[ok] - 404958790141
[尝试次数：1] - [剩余数量：1020] - [当前时间：21:10:06]

[ok] - 315352401871
[尝试次数：1] - [剩余数量：1019] - [当前时间：21:10:06]

[ok] - 315407297973
[尝试次数：1] - [剩余数量：1018] - [当前时间：21:10:07]

[ok] - 315341157493
[尝试次数：1] - [剩余数量：1017] - [当前时间：21:10:07]

[ok] - 315238974557
[尝试次数：1] - [剩余数量：1016] - [当前时间：21:10:07]

[ok] - 387040152752
[尝试次数：1] - [剩余数量：1015] - [当前时间：21:10:08]

[ok] - 315299197697
[尝试次数：1] - [剩余数量：1014] - [当前时间：21:10:08]

[ok] - 315405506676
[尝试次数：2] - [剩余数量：1013] - [当前时间：21:10:08]

[ok] - 314492858117
[尝试次数：1] - [剩余数量：1012] - [当前时间：21:10:09]

[ok] - 404940115091
[尝试次数：1] - [剩余数量：1011] - [当前时间：21:10:09]

[ok] - 386938821799
[尝试次数：1] - [剩余数量：1010] - [当前时间：21:10:09]

[ok] - 386837201879
[尝试次数：1] - [剩余数量：1009] - [当前时间：21:10:09]

[ok] - 386432598043
[尝试次数：1] - [剩余数量：1008] - [当前时间：21:10:09]

[ok] - 401948550125
[尝试次数：1] - [剩余数量：1007] - [当前时间：21:10:09]

[ok] - 315031724353
[尝试次数：2] - [剩余数量：1006] - [当前时间：21:10:10]

[ok] - 3

[ok] - 386887721800
[尝试次数：5] - [剩余数量：887] - [当前时间：21:10:48]

[ok] - 387014785409
[尝试次数：2] - [剩余数量：886] - [当前时间：21:10:48]

[ok] - 315192198139
[尝试次数：1] - [剩余数量：885] - [当前时间：21:10:49]

[ok] - 315405704169
[尝试次数：2] - [剩余数量：884] - [当前时间：21:10:49]

[ok] - 386979829205
[尝试次数：3] - [剩余数量：883] - [当前时间：21:10:49]

[ok] - 386977141768
[尝试次数：22] - [剩余数量：882] - [当前时间：21:10:50]

[ok] - 315325273930
[尝试次数：6] - [剩余数量：881] - [当前时间：21:10:50]

[ok] - 404888612932
[尝试次数：1] - [剩余数量：880] - [当前时间：21:10:50]

[ok] - 405012192703
[尝试次数：2] - [剩余数量：879] - [当前时间：21:10:50]

[ok] - 386930121887
[尝试次数：1] - [剩余数量：878] - [当前时间：21:10:51]

[ok] - 404842538655
[尝试次数：1] - [剩余数量：877] - [当前时间：21:10:51]

[ok] - 386860242282
[尝试次数：3] - [剩余数量：876] - [当前时间：21:10:51]

[ok] - 404885665881
[尝试次数：7] - [剩余数量：875] - [当前时间：21:10:51]

[ok] - 315257728278
[尝试次数：3] - [剩余数量：874] - [当前时间：21:10:53]

[ok] - 405005824029
[尝试次数：4] - [剩余数量：873] - [当前时间：21:10:53]

[ok] - 315275176312
[尝试次数：1] - [剩余数量：872] - [当前时间：21:10:53]

[ok] - 315208841750
[尝试

...

[ok] - 405003951867
[尝试次数：7] - [剩余数量：616] - [当前时间：21:12:17]

[ok] - 404938094406
[尝试次数：2] - [剩余数量：615] - [当前时间：21:12:17]

[ok] - 386962709516
[尝试次数：8] - [剩余数量：614] - [当前时间：21:12:17]

[ok] - 315325325993
[尝试次数：3] - [剩余数量：613] - [当前时间：21:12:18]

[ok] - 404842270347
[尝试次数：6] - [剩余数量：612] - [当前时间：21:12:18]

[ok] - 386882837427
[尝试次数：1] - [剩余数量：611] - [当前时间：21:12:18]

[ok] - 386910303334
[尝试次数：5] - [剩余数量：610] - [当前时间：21:12:18]

[ok] - 315283329919
[尝试次数：1] - [剩余数量：609] - [当前时间：21:12:19]

[ok] - 315381702051
[尝试次数：6] - [剩余数量：608] - [当前时间：21:12:20]

[ok] - 315400382886
[尝试次数：3] - [剩余数量：607] - [当前时间：21:12:20]

[ok] - 404888373275
[尝试次数：2] - [剩余数量：606] - [当前时间：21:12:20]

[ok] - 404938503504
[尝试次数：14] - [剩余数量：605] - [当前时间：21:12:20]

[ok] - 404888373216
[尝试次数：1] - [剩余数量：604] - [当前时间：21:12:20]

[ok] - 386991757018
[尝试次数：6] - [剩余数量：603] - [当前时间：21:12:20]

[ok] - 404888372551
[尝试次数：1] - [剩余数量：602] - [当前时间：21:12:21]

[ok] - 387008067290
[尝试次数：2] - [剩余数量：601] - [当前时间：21:12:21]

[ok] - 315293338851
[尝试

[ok] - 386860265434
[尝试次数：9] - [剩余数量：479] - [当前时间：21:13:03]

[ok] - 315225942279
[尝试次数：2] - [剩余数量：478] - [当前时间：21:13:04]

[ok] - 404893252683
[尝试次数：1] - [剩余数量：477] - [当前时间：21:13:04]

[ok] - 386859701895
[尝试次数：1] - [剩余数量：476] - [当前时间：21:13:05]

[ok] - 386910373864
[尝试次数：1] - [剩余数量：475] - [当前时间：21:13:06]

[ok] - 404996763601
[尝试次数：1] - [剩余数量：474] - [当前时间：21:13:06]

[ok] - 404862679489
[尝试次数：2] - [剩余数量：473] - [当前时间：21:13:07]

[ok] - 315407324825
[尝试次数：1] - [剩余数量：472] - [当前时间：21:13:07]

[ok] - 386887556322
[尝试次数：3] - [剩余数量：471] - [当前时间：21:13:07]

[ok] - 386860241085
[尝试次数：1] - [剩余数量：470] - [当前时间：21:13:07]

[ok] - 404880421299
[尝试次数：3] - [剩余数量：469] - [当前时间：21:13:07]

[ok] - 386894372587
[尝试次数：4] - [剩余数量：468] - [当前时间：21:13:07]

[ok] - 386860265623
[尝试次数：4] - [剩余数量：467] - [当前时间：21:13:08]

[ok] - 386860264487
[尝试次数：1] - [剩余数量：466] - [当前时间：21:13:09]

[ok] - 386836788173
[尝试次数：1] - [剩余数量：465] - [当前时间：21:13:09]

[ok] - 386837205264
[尝试次数：13] - [剩余数量：464] - [当前时间：21:13:10]

[ok] - 315404393113
[尝试

[ok] - 315400713118
[尝试次数：1] - [剩余数量：344] - [当前时间：21:13:50]

[ok] - 315257590025
[尝试次数：1] - [剩余数量：343] - [当前时间：21:13:50]

[ok] - 315236879456
[尝试次数：1] - [剩余数量：342] - [当前时间：21:13:51]

[ok] - 404943582651
[尝试次数：2] - [剩余数量：341] - [当前时间：21:13:51]

[ok] - 315365013334
[尝试次数：1] - [剩余数量：340] - [当前时间：21:13:51]

[ok] - 315347507721
[尝试次数：2] - [剩余数量：339] - [当前时间：21:13:52]

[ok] - 387040967563
[尝试次数：1] - [剩余数量：338] - [当前时间：21:13:52]

[ok] - 387040154053
[尝试次数：1] - [剩余数量：337] - [当前时间：21:13:52]

[ok] - 387003680334
[尝试次数：1] - [剩余数量：336] - [当前时间：21:13:53]

[ok] - 387004078576
[尝试次数：2] - [剩余数量：335] - [当前时间：21:13:54]

[ok] - 404888373398
[尝试次数：2] - [剩余数量：334] - [当前时间：21:13:54]

[ok] - 315397929561
[尝试次数：1] - [剩余数量：333] - [当前时间：21:13:54]

[ok] - 315225899528
[尝试次数：1] - [剩余数量：332] - [当前时间：21:13:54]

[ok] - 315225904041
[尝试次数：1] - [剩余数量：331] - [当前时间：21:13:55]

[ok] - 315225903810
[尝试次数：1] - [剩余数量：330] - [当前时间：21:13:56]

[ok] - 386889921096
[尝试次数：1] - [剩余数量：329] - [当前时间：21:13:56]

[ok] - 386889944091
[尝试次

[ok] - 315225903288
[尝试次数：4] - [剩余数量：208] - [当前时间：21:14:35]

[ok] - 404888373836
[尝试次数：1] - [剩余数量：207] - [当前时间：21:14:35]

[ok] - 315253582598
[尝试次数：1] - [剩余数量：206] - [当前时间：21:14:35]

[ok] - 315278989451
[尝试次数：2] - [剩余数量：205] - [当前时间：21:14:35]

[ok] - 404883394734
[尝试次数：1] - [剩余数量：204] - [当前时间：21:14:36]

[ok] - 387007835142
[尝试次数：1] - [剩余数量：203] - [当前时间：21:14:36]

[ok] - 404842538782
[尝试次数：1] - [剩余数量：202] - [当前时间：21:14:37]

[ok] - 404976565984
[尝试次数：1] - [剩余数量：201] - [当前时间：21:14:38]

[ok] - 404876772860
[尝试次数：1] - [剩余数量：200] - [当前时间：21:14:39]

[ok] - 315384008211
[尝试次数：1] - [剩余数量：199] - [当前时间：21:14:39]

[ok] - 386857362903
[尝试次数：1] - [剩余数量：198] - [当前时间：21:14:39]

[ok] - 386965006751
[尝试次数：1] - [剩余数量：197] - [当前时间：21:14:40]

[ok] - 386894235745
[尝试次数：2] - [剩余数量：196] - [当前时间：21:14:40]

[ok] - 386889941692
[尝试次数：1] - [剩余数量：195] - [当前时间：21:14:40]

[ok] - 387028928262
[尝试次数：1] - [剩余数量：194] - [当前时间：21:14:41]

[ok] - 404846702790
[尝试次数：1] - [剩余数量：193] - [当前时间：21:14:41]

[ok] - 315257409982
[尝试次

[ok] - 315336130490
[尝试次数：3] - [剩余数量：73] - [当前时间：21:15:19]

[ok] - 315333928908
[尝试次数：1] - [剩余数量：72] - [当前时间：21:15:20]

[ok] - 315257589123
[尝试次数：25] - [剩余数量：71] - [当前时间：21:15:20]

[ok] - 315397889345
[尝试次数：1] - [剩余数量：70] - [当前时间：21:15:20]

[ok] - 404883394605
[尝试次数：1] - [剩余数量：69] - [当前时间：21:15:21]

[ok] - 315400713049
[尝试次数：2] - [剩余数量：68] - [当前时间：21:15:21]

[ok] - 315368213071
[尝试次数：6] - [剩余数量：67] - [当前时间：21:15:21]

[ok] - 315253582410
[尝试次数：2] - [剩余数量：66] - [当前时间：21:15:21]

[ok] - 404891776976
[尝试次数：1] - [剩余数量：65] - [当前时间：21:15:21]

[ok] - 386993847049
[尝试次数：1] - [剩余数量：64] - [当前时间：21:15:22]

[ok] - 386873693097
[尝试次数：10] - [剩余数量：63] - [当前时间：21:15:22]

[ok] - 315257588884
[尝试次数：7] - [剩余数量：62] - [当前时间：21:15:23]

[ok] - 315249045184
[尝试次数：1] - [剩余数量：61] - [当前时间：21:15:23]

[ok] - 315269968318
[尝试次数：9] - [剩余数量：60] - [当前时间：21:15:23]

[ok] - 315384004780
[尝试次数：1] - [剩余数量：59] - [当前时间：21:15:24]

[ok] - 404883394637
[尝试次数：3] - [剩余数量：58] - [当前时间：21:15:24]

[ok] - 404965390356
[尝试次数：2] - [剩余数量：5